Installing Dependencies

In [8]:
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Setting Up Selenium

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# ---- Selenium setup ----
options = Options()
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)...")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

# ---- Category list (add as many as you want) ----
categories = {
    "women-dresses": "https://www.myntra.com/women-dresses",
    "women-kurtis": "https://www.myntra.com/women-kurtis",
    "men-shoes": "https://www.myntra.com/men-shoes",
    "women-tops": "https://www.myntra.com/women-tops",
    # add more categories here
}

# ---- Example: scrape the first category for now ----
for cat_name, url in categories.items():
    print(f"Scraping category: {cat_name} from {url}")
    
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    # Wait for dynamic content
    WebDriverWait(driver, 40).until(
        EC.presence_of_element_located((By.CLASS_NAME, "product-base"))
    )
    html = driver.page_source

    # TODO: parse products + save to CSV (next block)
    


Scraping category: women-dresses from https://www.myntra.com/women-dresses
Scraping category: women-kurtis from https://www.myntra.com/women-kurtis


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000104994c20 cxxbridge1$str$ptr + 2747652
1   chromedriver                        0x000000010498cb20 cxxbridge1$str$ptr + 2714628
2   chromedriver                        0x00000001044d50c8 cxxbridge1$string$len + 90536
3   chromedriver                        0x000000010451c618 cxxbridge1$string$len + 382712
4   chromedriver                        0x000000010455d99c cxxbridge1$string$len + 649852
5   chromedriver                        0x0000000104510934 cxxbridge1$string$len + 334356
6   chromedriver                        0x000000010495788c cxxbridge1$str$ptr + 2496880
7   chromedriver                        0x000000010495aab8 cxxbridge1$str$ptr + 2509724
8   chromedriver                        0x0000000104938510 cxxbridge1$str$ptr + 2369012
9   chromedriver                        0x000000010495b360 cxxbridge1$str$ptr + 2511940
10  chromedriver                        0x0000000104929610 cxxbridge1$str

In [ ]:
from bs4 import BeautifulSoup

for cat_name, url in categories.items():
    print(f"Scraping category: {cat_name} from {url}")
    
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "product-base"))
    )
    html = driver.page_source
    
    # ---- Parse the HTML for this category ----
    soup = BeautifulSoup(html, 'html.parser')
    
    # TODO: extract product info + save
    # Example: products = soup.find_all("li", {"class": "product-base"})
    
    break  # remove later when you’re ready to scrape all categories


Scraping category: women-dresses from https://www.myntra.com/women-dresses


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001007b4c20 cxxbridge1$str$ptr + 2747652
1   chromedriver                        0x00000001007acb20 cxxbridge1$str$ptr + 2714628
2   chromedriver                        0x00000001002f50c8 cxxbridge1$string$len + 90536
3   chromedriver                        0x000000010033c618 cxxbridge1$string$len + 382712
4   chromedriver                        0x000000010037d99c cxxbridge1$string$len + 649852
5   chromedriver                        0x0000000100330934 cxxbridge1$string$len + 334356
6   chromedriver                        0x000000010077788c cxxbridge1$str$ptr + 2496880
7   chromedriver                        0x000000010077aab8 cxxbridge1$str$ptr + 2509724
8   chromedriver                        0x0000000100758510 cxxbridge1$str$ptr + 2369012
9   chromedriver                        0x000000010077b360 cxxbridge1$str$ptr + 2511940
10  chromedriver                        0x0000000100749610 cxxbridge1$str

Extracting Product Information

In [ ]:
all_data = []  # master list for everything

for cat_name, url in categories.items():
    print(f"\nScraping category: {cat_name} from {url}")
    
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "product-base"))
    )
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    products = soup.find_all('li', {'class': 'product-base'})
    print(f"Found {len(products)} products in {cat_name}")

    # Scraped data for this category
    data = []

    for item in products:
        brand = item.find('h3', {'class': 'product-brand'}).text.strip()
        name = item.find('h4', {'class': 'product-product'}).text.strip()

        # Price details
        price_elements = item.find('div', {'class': 'product-price'})
        prices = price_elements.find_all('span') if price_elements else []
        discounted_price = prices[0].text.strip() if len(prices) > 0 else None
        original_price = prices[1].text.strip() if len(prices) > 1 else None
        discount = prices[2].text.strip() if len(prices) > 2 else None

        # Product Link
        link_tag = item.find('a', href=True)
        product_link = "https://www.myntra.com" + link_tag['href'] if link_tag else None

        data.append({
            'Category': cat_name,
            'Brand': brand,
            'Name': name,
            'Discounted Price': discounted_price,
            'Original Price': original_price,
            'Discount': discount,
            'Link': product_link
        })

    # Add this category's data to master list
    all_data.extend(data)

    # Save category CSV immediately (optional)
    pd.DataFrame(data).to_csv(f"myntra_{cat_name}.csv", index=False)
    print(f"Saved {cat_name} to myntra_{cat_name}.csv")


Found 50 products


In [ ]:
import pandas as pd

# Example: assume `all_data` is a dict with keys = category names, values = lists of product dicts
# all_data = {
#     "womens_dresses": data_for_dresses,
#     "mens_shoes": data_for_shoes,
#     ...
# }

dfs = {}  # store DataFrames by category
for category, data in all_data.items():
    dfs[category] = pd.DataFrame(data)
    print(f"\nPreview for {category}:")
    print(dfs[category].head())


     Brand                                 Name Discounted Price  \
0   Claura        Women  Fit & Flare Midi Dress  Rs. 724Rs. 2899   
1   anayna           Women Fit & Flared Dresses  Rs. 883Rs. 3840   
2    MANGO         Round Neck Midi A-Line Dress         Rs. 5790   
3   Stylum  Floral Print Fit & Flare Maxi Dress  Rs. 739Rs. 2899   
4  Varanga                    A-Line Maxi Dress  Rs. 749Rs. 2999   

  Original Price  Discount                                               Link  
0        Rs. 724  Rs. 2899  https://www.myntra.comdresses/claura/claura-wo...  
1        Rs. 883  Rs. 3840  https://www.myntra.comdresses/anayna/anayna-wo...  
2           None      None  https://www.myntra.comdresses/mango/mango-roun...  
3        Rs. 739  Rs. 2899  https://www.myntra.comdresses/stylum/stylum-fl...  
4        Rs. 749  Rs. 2999  https://www.myntra.comdresses/varanga/varanga-...  


Cleaning the Data

In [ ]:
import re

def extract_prices(price_str):
    if pd.isna(price_str):
        return None, None
    prices = re.findall(r'Rs\.?\s?(\d+)', price_str.replace(",", ""))
    if len(prices) == 2:
        return int(prices[0]), int(prices[1])
    elif len(prices) == 1:
        return int(prices[0]), None
    else:
        return None, None

cleaned_dfs = {}

for category, df in dfs.items():
    print(f"\nCleaning data for {category}...")
    
    # Strip extra spaces from column names
    df.columns = df.columns.str.strip()
    
    # Split discounted and original prices
    df[['Discounted_Price', 'Original_Price']] = df['Discounted Price'].apply(
        lambda x: pd.Series(extract_prices(x))
    )
    
    # Fix broken URLs (catching multiple cases if needed)
    df['Link'] = df['Link'].str.replace("www.myntra.comdresses", "www.myntra.com/dresses", regex=False)
    
    # Calculate discount percentage safely
    df['Discount_Percentage'] = (
        (df['Original_Price'] - df['Discounted_Price']) / df['Original_Price'] * 100
    ).round(1)
    
    # Drop old columns
    df = df.drop(columns=['Discounted Price', 'Original Price', 'Discount'])
    
    # Drop rows with missing prices
    df = df.dropna(subset=['Discounted_Price'])
    
    # Reset index
    df = df.reset_index(drop=True)
    
    cleaned_dfs[category] = df
    print(df.head())


     Brand                                 Name  \
0   Claura        Women  Fit & Flare Midi Dress   
1   anayna           Women Fit & Flared Dresses   
2    MANGO         Round Neck Midi A-Line Dress   
3   Stylum  Floral Print Fit & Flare Maxi Dress   
4  Varanga                    A-Line Maxi Dress   

                                                Link  Discounted_Price  \
0  https://www.myntra.com/dresses/claura/claura-w...             724.0   
1  https://www.myntra.com/dresses/anayna/anayna-w...             883.0   
2  https://www.myntra.com/dresses/mango/mango-rou...            5790.0   
3  https://www.myntra.com/dresses/stylum/stylum-f...             739.0   
4  https://www.myntra.com/dresses/varanga/varanga...             749.0   

   Original_Price  Discount_Percentage  
0          2899.0                 75.0  
1          3840.0                 77.0  
2             NaN                  NaN  
3          2899.0                 74.5  
4          2999.0                 75.0  


In [ ]:
import os  

for category, data in category_data.items():  
    df = pd.DataFrame(data)
    file_path = os.path.join(output_dir, f"myntra_{category}.csv")
    df.to_csv(file_path, index=False)
    print(f"Saved {category} → {file_path}")